In [1]:
import os
import json
import time
import base64
from pathlib import Path
from google.cloud import texttospeech_v1beta1
from IPython.display import clear_output as clear

In [2]:
language = 'portuguese'

In [3]:
path = os.path.abspath('').replace('scripts', '')
top_dir = Path(path).parent.absolute()
data_dir = os.path.join(top_dir, 'conjugations', 'data')

In [4]:
# for verb in conjugations:
#     if len(conjugations[verb]['compound']['indicative']['future']['ich'].split(' ')) < 3:
#         if 'habe' in conjugations[verb]['compound']['indicative']['present']['ich'].split(' '):
#             auxillary = 'haben'
#         else:
#             auxillary = "sein"

#         for subject in conjugations[verb]['compound']['indicative']['future']:
#             conjugations[verb]['compound']['indicative']['future'][subject] += ' ' + auxillary
#             conjugations[verb]['compound']['subjunctive']['conditional'][subject] += ' ' + auxillary

In [5]:
# for verb in conjugations:
#     for subject in conjugations[verb]['compound']['subjunctive']['future']:
#         werden = conjugations[verb]['compound']['subjunctive']['future'][subject].split(' ')[0]
#         participle = conjugations[verb]['compound']['indicative']['future'][subject].split(' ')[1]
#         auxillary = conjugations[verb]['compound']['indicative']['future'][subject].split(' ')[2]

#         conjugations[verb]['compound']['subjunctive']['future'][subject] = ' '.join([werden, participle, auxillary])

In [6]:
# f = os.path.join(data_dir, 'language-specific', 'conjugations_german.json')
# with open(f, "w", encoding = 'utf8') as file:
#     json.dump(conjugations, file, indent = 4, ensure_ascii = False)

In [7]:
with open(os.path.join(data_dir, 'language-specific', f'conjugations_{language}_new.json'), 'r', encoding = 'utf8') as file:
    new = json.loads(file.read())

with open(os.path.join(data_dir, 'language-specific', f'conjugations_{language}.json'), 'r', encoding = 'utf8') as file:
    old = json.loads(file.read())

In [8]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(top_dir, 'audio', 'data', 'gckey.json')

In [9]:
class nestedDict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()
        return value

In [10]:
def ttsSynthesize(phrase, language_code, voice_name):
    client = texttospeech_v1beta1.TextToSpeechClient()

    input = texttospeech_v1beta1.SynthesisInput()
    input.text = phrase

    voice = texttospeech_v1beta1.VoiceSelectionParams()
    voice.language_code = language_code
    voice.name = voice_name

    audio_config = texttospeech_v1beta1.AudioConfig()
    audio_config.audio_encoding = "MP3_64_KBPS"
    audio_config.sample_rate_hertz = 44100

    request = texttospeech_v1beta1.SynthesizeSpeechRequest(
        input = input,
        voice = voice,
        audio_config = audio_config,
    )

    response = client.synthesize_speech(request = request)

    return str(base64.b64encode(response.audio_content))[2:-1]

In [11]:
voices = {
    'spanish': 'es-ES-Wavenet-B',
    'french': 'fr-FR-Wavenet-D',
    'german': 'de-DE-Wavenet-B',
    'italian': 'it-IT-Wavenet-C',
    'portuguese': 'pt-PT-Wavenet-B',
}

language_codes = {
    'spanish': 'es_ES',
    'french': 'fr_FR',
    'german': 'de_DE',
    'italian': 'it_IT',
    'portuguese': 'pt_PT'
}

In [12]:
i = 0
changes = []

In [13]:
with open(os.path.join(top_dir, 'audio', 'out', f'audio_{language}.json'), 'r', encoding = 'utf8') as file:
    audio = nestedDict(json.loads(file.read()))

In [14]:
for verb in new:
    if new[verb]['rank'] < 2001:
        for complexity in ['simple', 'compound']:
            for mood in new[verb][complexity]:
                for tense in new[verb][complexity][mood]:
                    for subject in new[verb][complexity][mood][tense]:
                        if new[verb][complexity][mood][tense][subject] != old[verb][complexity][mood][tense][subject]:
                            if [verb, complexity, mood, tense] not in changes:
                                changes.append([verb, complexity, mood, tense])

In [15]:
while i < len(changes):

    try:
        verb, complexity, mood, tense = changes[i]
        collated = {}

        for subject in new[verb][complexity][mood][tense]:
        
            conjugation = new[verb][complexity][mood][tense][subject]

            if conjugation not in collated.keys():
                collated[conjugation] = ttsSynthesize(conjugation, language_codes[language], voices[language])

        audio[verb][complexity][mood][tense] = collated
        i += 1

        clear(); print(f'{verb} – {round(i * 100/ len(changes), 3)}%')

    except Exception as ex:
        clear(); print(f'ERROR {ex} – SLEEEPING FOR 2 MINUTES')
        time.sleep(120)

publicitar – 100.0%


In [16]:
f = os.path.join(top_dir, 'audio', 'out', f'audio_{language}.json')

with open(f, "w", encoding = 'utf8') as file:
    json.dump(audio, file, indent = 4, ensure_ascii = False)